In [1]:
import numpy as np
import pandas as pd
import polars as pl
MAIN_PATH = "../data"

In [2]:
corpus = (
    pl.read_parquet(f'{MAIN_PATH}/corpus-240306.parquet')
    .filter(~pl.col('title').str.contains('(survey|benchmark|review|tutorial)'))
)

In [3]:
def get_pivot_table(df, column=None, variable=None):
    metric = f'is_{column}_success'
    df[metric] = df[f'{column}_success'] > 0
    
    # pivot table
    inner_tables = []
    for is_success, group in df.groupby(metric):
        variable_values, freqs = np.unique(
            np.concatenate(
                group[variable.title()].apply(np.unique).values
            ),
            return_counts=True
        )
        inner_tables.append(
            pd.DataFrame({
                variable: variable_values,
                'freq': freqs,
                metric: is_success
            })
        )

    pivoted_table = pd.concat(inner_tables).pivot_table(
        index=variable,
        columns=metric,
        values='freq',
        fill_value=0
    )
    return pivoted_table

In [ ]:
fc_inst = (
    pl.read_parquet(f'{MAIN_PATH}/papers_with_info_filtered5.parquet')
    .unique('corpusid')
    .filter(pl.col('venue_area').ne('Not-AI'))
    .select('corpusid', 'venue_area', 'Institutions')
)

In [6]:
inst = pd.read_csv(f'{MAIN_PATH}/institutions-metadata.csv', names=[
    'institutions_names',
    'institutions_ids',
    'institutions_ror',
    'locations',
    'doi',
    'corpusid',
])

In [7]:
combined_inst = pl.concat([
    (
        fc_inst.join(corpus.select('corpusid', 'year_min_innovation', 'year_min_development'), how='left', on='corpusid')
        .select('corpusid', 'year_min_innovation', 'year_min_development', 'venue_area', pl.col('Institutions').alias('institutions'))
    ),
    (
        pl.from_dataframe(inst)
        .with_columns(pl.col('corpusid').cast(int))
        .join(
            corpus,
            how='inner',
            on='corpusid'
        )
        .select('corpusid', 'year_min_innovation', 'year_min_development', 'venue_area', pl.col('institutions_names').map_elements(eval).alias('institutions'))
    )
])

In [8]:
venue_map = {
    'impactful': 'Top General Science',
    'average': 'Computer Science',
    'ai': 'AI',
    'hci': 'HCI',
    'linguistic': 'NLP',
    'information': 'Database',
    'mining': 'Data Mining',
    'vision': 'Computer Vision',
}

In [ ]:
df = combined_inst.filter(pl.col('venue_area').is_in(['impactful', 'average']).not_()).to_pandas()

df["is_innovation_success"] = df["year_min_innovation"].notnull()
df["is_development_success"] = df["year_min_development"].notnull()
df = df[['corpusid', 'venue_area', 'is_innovation_success', 'is_development_success', 'institutions']].explode('institutions')
df = df.drop_duplicates(subset=['corpusid', 'institutions']).dropna(subset='institutions')
df["is_either_success"] = df[['is_innovation_success', 'is_development_success']].sum(axis=1).gt(0)
df['venue_area'] = df['venue_area'].map(venue_map)

df_agg = df.groupby(['institutions', 'venue_area']).agg({
    'is_innovation_success': 'sum',
    'is_development_success': 'sum',
    'is_either_success': 'sum',
    'corpusid': 'count',
})

In [10]:
df_agg.sum(axis=0)

is_innovation_success      19028
is_development_success     14004
is_either_success          31924
corpusid                  242970
dtype: int64

In [ ]:
from scipy.stats import beta

df_agg['impact_innovation'] = beta(2 + df_agg['is_innovation_success'], 23 + df_agg['corpusid'] - df_agg['is_innovation_success']).ppf(0.05)
df_agg['impact_development'] = beta(2 + df_agg['is_development_success'], 23 + df_agg['corpusid'] - df_agg['is_development_success']).ppf(0.05)

In [12]:
nb_papers = df_agg.reset_index().groupby('institutions').corpusid.sum()
top_institutions = nb_papers.nlargest(50).index.tolist()

In [14]:
impacts = df_agg.reset_index().pivot_table(
    index='institutions',
    columns='venue_area',
    values=['impact_innovation', 'impact_development']
).loc[top_institutions]

In [15]:
top25 = df_agg.reset_index().groupby('institutions').agg({
    'impact_innovation': 'sum',
    'impact_development': 'sum'
}).sum(axis=1).divide(12)[top_institutions].nlargest(25).reset_index()

top25.columns = ['institutions', 'avg_impact']

## Table 3

In [16]:
print(top25.set_index('institutions').style.format('{:.1%}').to_latex())

\begin{tabular}{lr}
 & avg_impact \\
institutions &  \\
Google (United States) & 12.6% \\
Microsoft (United States) & 11.8% \\
Stanford University & 11.4% \\
Microsoft Research Asia (China) & 10.9% \\
Massachusetts Institute of Technology & 10.8% \\
Carnegie Mellon University & 10.7% \\
University of California, Berkeley & 10.1% \\
Cornell University & 9.0% \\
ETH Zurich & 9.0% \\
Microsoft Research (United Kingdom) & 8.8% \\
University of Washington & 8.7% \\
University of Illinois Urbana-Champaign & 8.5% \\
Georgia Institute of Technology & 8.3% \\
Tencent (China) & 7.9% \\
IBM (United States) & 7.9% \\
University of Massachusetts Amherst & 7.6% \\
Chinese University of Hong Kong & 7.6% \\
University of Michigan–Ann Arbor & 7.2% \\
Tsinghua University & 7.2% \\
Alibaba Group (China) & 6.8% \\
University of Maryland, College Park & 6.8% \\
Fudan University & 6.7% \\
Shanghai Jiao Tong University & 6.5% \\
University of Science and Technology of China & 6.5% \\
Zhejiang University & 6.

In [34]:
print(
    pd.concat([
        top25.set_index('institutions'),
        impacts.loc[top25.institutions, ['impact_innovation', 'impact_development']]
    ], axis=1)
    .style.format('{:.1%}')
    .highlight_max(axis=0, props='font-weight:bold;')
    .to_latex(convert_css=True)
)

\begin{tabular}{lrrrrrrrrrrrrr}
 & avg_impact & ('impact_innovation', 'AI') & ('impact_innovation', 'Computer Vision') & ('impact_innovation', 'Data Mining') & ('impact_innovation', 'Database') & ('impact_innovation', 'HCI') & ('impact_innovation', 'NLP') & ('impact_development', 'AI') & ('impact_development', 'Computer Vision') & ('impact_development', 'Data Mining') & ('impact_development', 'Database') & ('impact_development', 'HCI') & ('impact_development', 'NLP') \\
institutions &  &  &  &  &  &  &  &  &  &  &  &  &  \\
Google (United States) & \bfseries 12.6% & 5.7% & 15.7% & 11.9% & 14.9% & 8.6% & 9.5% & 12.3% & \bfseries 36.7% & 6.8% & 2.9% & 0.9% & 25.1% \\
Microsoft (United States) & 11.8% & 7.2% & 24.9% & 15.7% & 18.5% & \bfseries 19.6% & 17.8% & 2.3% & 12.6% & 5.2% & 2.3% & 0.3% & 15.4% \\
Stanford University & 11.4% & \bfseries 8.7% & 23.9% & 14.6% & 16.6% & 7.8% & 15.5% & 12.3% & 13.4% & 11.9% & 3.5% & 0.7% & 8.1% \\
Microsoft Research Asia (China) & 10.9% & 8.3% & 22.8% &